# <u> LSTM ENCODER DECODER

# Data

In [1]:
import utils
import numpy as np

#utils.setup_nltk()
PRESIDENT = 'reagan'
speeches = utils.read_all_text_files(PRESIDENT)

### Preprocessing

In [2]:
filter_list = [':', '(', ')', ',', '-',]
filtered_speeches = []

for speech in speeches:
    filtered_speech = []
    for word in speech:
        # filter out unwanted words
        if word not in filter_list:
            # lower word
            filtered_speech.append(word.lower())
    filtered_speeches.append(filtered_speech)

### Create N-Grams

In [3]:
from nltk import ngrams

WINDOW = 5
grams = [ngrams(s, WINDOW+1) for s in filtered_speeches]
flat_grams = [ng for speech in grams for ng in speech]

### Split Data

In [4]:
import pandas as pd

X = [' '.join(list(x[0:WINDOW])) for x in flat_grams]
Y = [x[-1] for x in flat_grams]
df = pd.DataFrame.from_dict({'x':X, 'y':Y})

# persist
csv_name = '../data/lstm/preproc/{}_encdec_{}grams.csv'.format(PRESIDENT, str(WINDOW))
df.to_csv(csv_name, index=False)

In [5]:
import torchtext

XFIELD = torchtext.data.Field(sequential=True)
YFIELD = torchtext.data.Field(sequential=True)
DATA = torchtext.data.TabularDataset(csv_name,'csv', 
                                     [('x', XFIELD),('y', YFIELD)], skip_header=True)

XFIELD.build_vocab(DATA)  
YFIELD.build_vocab(DATA)

/home/gabriel/.local/lib/python3.8/site-packages/torchtext/data/field.py:150: UserWarning: Field class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)
/home/gabriel/.local/lib/python3.8/site-packages/torchtext/data/example.py:68: UserWarning: Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.', UserWarning)
/home/gabriel/.local/lib/python3.8/site-packages/torchtext/data/example.py:78: UserWarning: Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes f

In [6]:
from torchtext.data import BucketIterator, Iterator
import torch

BATCH_SIZE = 64
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
train_iterator = Iterator(DATA, BATCH_SIZE, device=device, train=True)

/home/gabriel/.local/lib/python3.8/site-packages/torchtext/data/iterator.py:48: UserWarning: Iterator class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


In [7]:
assert device.type == 'cuda'

# Neural Networks

### Encoder

In [8]:
import torch.nn as nn

class Encoder(nn.Module):
    def __init__(self, vocab_size, hidden_size, embedding_dim, num_layers):
        super(Encoder, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_size, num_layers=num_layers, bidirectional=True)

    def forward(self, x, h0, c0):
        x = self.embedding(x).unsqueeze(0)
        out, (h0, c0) = self.lstm(x, (h0, c0))
        return out, (h0, c0)

### Decoder

In [9]:
class Decoder(nn.Module):
    def __init__(self, vocab_size, hidden_size, embedding_dim, num_layers):
        super(Decoder, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_size, num_layers=num_layers, dropout=0.5, bidirectional=True)
        self.dense = nn.Linear(hidden_size*2, vocab_size)
        self.softmax = nn.LogSoftmax(dim=1)
  
    def forward(self, x, h0, c0):
        x = self.embedding(x)
        x, (h0, c0) = self.lstm(x, (h0, c0))
        x = self.dense(x.squeeze(0))
        x = self.softmax(x)
        return x, (h0, c0)

### Training

In [10]:
HIDDEN_SIZE = 20
EMBEDDING_SIZE = 50
NUM_LAYERS = 2
LR = 0.01
ENC_LEARNING_RATE = LR
DEC_LEARNING_RATE = LR
criterion = nn.NLLLoss()

encoder = Encoder(len(XFIELD.vocab), HIDDEN_SIZE, EMBEDDING_SIZE, NUM_LAYERS).to(device)
decoder = Decoder(len(XFIELD.vocab), HIDDEN_SIZE, EMBEDDING_SIZE, NUM_LAYERS).to(device)
enc_optimizer = torch.optim.Adam(encoder.parameters(), lr = ENC_LEARNING_RATE)
dec_optimizer = torch.optim.Adam(decoder.parameters(), lr = DEC_LEARNING_RATE)

In [11]:
from tqdm import tqdm

EPOCHS = 5
for ep in range(EPOCHS):
    ep_loss = 0
    
    for batch in tqdm(train_iterator):
        if len(batch) != BATCH_SIZE: break;
        inp = batch.x
        target = batch.y
        
        # init
        loss = 0
        h0 = torch.zeros(NUM_LAYERS*2, BATCH_SIZE, HIDDEN_SIZE).to(device)
        c0 = torch.zeros(NUM_LAYERS*2, BATCH_SIZE, HIDDEN_SIZE).to(device)
        enc_optimizer.zero_grad()
        dec_optimizer.zero_grad()
        
        # encode
        for w in range(inp.size(0)):
            enc_out, (h0, c0) = encoder(inp[w], h0, c0)
            
        # decode
        cur = inp[WINDOW-1].unsqueeze(0)
        dec_out, (_, _) = decoder(cur, h0, c0)        
        cur = torch.argmax(dec_out,dim=1)
        
        # loss
        # target_onehot = torch.nn.functional.one_hot(target.squeeze(), len(YFIELD.vocab))
        loss += criterion(dec_out, target.squeeze())
        
        # optimize
        ep_loss += loss
        loss.backward()
        enc_optimizer.step()
        dec_optimizer.step()
        
    print('AVG_LOSS={}, (ABS={})'.format(round((ep_loss/(len(DATA)/BATCH_SIZE)).item(),4), round(ep_loss.item(),2)))

  0%|          | 0/3591 [00:00<?, ?it/s]/home/gabriel/.local/lib/python3.8/site-packages/torchtext/data/batch.py:23: UserWarning: Batch class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)
  0%|          | 0/3591 [00:00<?, ?it/s]

AVG_LOSS=6.1319, (ABS=22018.8)


  0%|          | 0/3591 [00:00<?, ?it/s]

AVG_LOSS=5.6112, (ABS=20149.02)


  0%|          | 0/3591 [00:00<?, ?it/s]

AVG_LOSS=5.4086, (ABS=19421.51)


  0%|          | 0/3591 [00:00<?, ?it/s]

AVG_LOSS=5.2703, (ABS=18924.72)


100%|█████████▉| 3590/3591 [01:01<00:00, 58.64it/s]

AVG_LOSS=5.1723, (ABS=18572.9)


# Generate Text!

### functions

In [12]:
from torch import torch

def voc_index(words):
    return torch.tensor([XFIELD.vocab.stoi[x] for x in words]).to(device)

def predict(inp, RND_FACTOR=0, multiply=False, h0=None, c0=None):
    with torch.no_grad():
        
        if h0 == None:
            h0 = torch.zeros(2*NUM_LAYERS, BATCH_SIZE, HIDDEN_SIZE).to(device)
        if c0 == None:
            c0 = torch.zeros(2*NUM_LAYERS, BATCH_SIZE, HIDDEN_SIZE).to(device)
        
        for w in range(inp.size(0)):
                enc_out, (h0, c0) = encoder(inp[w], h0, c0)

        cur = inp[WINDOW-1].unsqueeze(0)
        dec_out, (h0, c0) = decoder(cur, h0, c0)
        
        # randomize
        if multiply:
            rnd = torch.rand(dec_out.shape).to(device) * RND_FACTOR + 1
            cur = torch.argmax(dec_out * rnd,dim=1)
        else:
            rnd = torch.rand(dec_out.shape).to(device) * RND_FACTOR
            cur = torch.argmax(dec_out.add(rnd),dim=1)

        return YFIELD.vocab.itos[cur[0].item()], (h0, c0)

def generate(intro=['good', 'evening', 'ladies', 'and', 'gentlemen'], multiply=False, rnd_factor=10, length=100, decay=None):
    text = intro
    h0 = torch.zeros(2*NUM_LAYERS, BATCH_SIZE, HIDDEN_SIZE).to(device)
    c0 = torch.zeros(2*NUM_LAYERS, BATCH_SIZE, HIDDEN_SIZE).to(device)
    
    for i in range(length):
        cur_window = text[-WINDOW:]
        vecs = voc_index(cur_window).view(WINDOW,1).repeat(1,BATCH_SIZE)
        
        if decay:
            prediction, (h0, c0) = predict(vecs, rnd_factor, multiply, h0, c0)
            #h0 = torch.rand(h0.shape).to(device) * h0
            #c0 = torch.rand(c0.shape).to(device) * c0
        else:
            prediction, _ = predict(vecs, rnd_factor, multiply)
            
        text.append(prediction)

    return ' '.join(text)

### create N speeches

In [13]:
%%time

generated = []
GENERATE_N = 10
MEAN_OG_SPEECH_LEN = round(np.mean([len(s) for s in speeches]))
for i in range(GENERATE_N):
    generated.append(
        generate(intro=['good', 'evening', 'ladies', 'and', 'gentlemen'], 
                 multiply=True, 
                 rnd_factor=1.2, 
                 length=MEAN_OG_SPEECH_LEN, 
                 decay=True)
    )

generated[0]

CPU times: user 7min 31s, sys: 5.51 s, total: 7min 36s
Wall time: 7min 36s


"good evening ladies and gentlemen the soviet union . and i 've been a few years . and we 're going to be done . and they 're going to be in the united states . the last year has been to the same time in the first time . the american people are going to be a little of nuclear weapons . and we must not not be a little of the american people . and we 're going to the united states . and finally the president . and i want to say the president 's voices . the american people were the world of the united states has been to be able to be done . the people is the same time . the president . the american people were not a new life . but i think that the president . the president . and i want to say that the american people are the same time . but the president . we 're going to the world of the world of the soviet union are going to the people . i think that we will not be no longer . and we 're going to be in the most time of the soviet union . and i think that the united states is the americ

# Evaluate

In [14]:
%%time

import metrics

# persist
for i in range(len(generated)):
    with open("../data/lstm/{}_generated/{}.txt".format(PRESIDENT, str(i)), "w") as text_file:
        text_file.write(generated[i]) 

# scores
mean_cos, std_cos, cos_sim = metrics.get_cosine_sim_tfidf(PRESIDENT, "lstm/{}_generated".format(PRESIDENT), print_results=True)
rouge = metrics.get_rouge_score(PRESIDENT, "lstm/{}_generated".format(PRESIDENT), print_results=True)
og_sen_len = metrics.calculate_mean_sentence_length(PRESIDENT)
gen_sen_len = metrics.calculate_mean_sentence_length("lstm/{}_generated".format(PRESIDENT))
print('mean sentence len (OG vs GEN) {}'.format((og_sen_len, gen_sen_len)))
print('mean sentence len diff {}'.format(og_sen_len-gen_sen_len))
og_w_len = metrics.calculate_mean_word_length(PRESIDENT)
gen_w_len = metrics.calculate_mean_word_length("lstm/{}_generated".format(PRESIDENT))
print('mean word len (OG vs GEN) {}'.format((og_w_len, gen_w_len)))
print('mean word len diff {}'.format(og_w_len-gen_w_len))
top = 15
rank_dist = metrics.get_top_n_rank_distance(orig_speeches_loc=PRESIDENT, gen_speeches_loc="lstm/{}_generated".format(PRESIDENT), n=top)
print('top{} rank distance {}'.format(top, rank_dist))

mean cosine similarity over all generated speeches: 0.040750365991311155
standard deviation of cosine similarity over all generated speeches: 0.002237969629810559
mean rouge score for all generated speeches: 0.34651980466404975
standard deviation of rouge score for all generated speeches: 0.0040786438457225
mean sentence len (OG vs GEN) (18.456016597510374, 9.771817794806301)
mean sentence len diff 8.684198802704072
mean word len (OG vs GEN) (4.7203044330963335, 3.7169796006709594)
mean word len diff 1.003324832425374
top15 rank distance 326.73236366266076
CPU times: user 53.5 s, sys: 110 ms, total: 53.6 s
Wall time: 53.6 s


# Save Models